In [ ]:
import numpy as np
import pandas as pd

# ---------- constants (cgs) ----------
c = 2.99792458e10              # cm/s
a_rad = 7.5657e-15             # erg cm^-3 K^-4
kB_erg_per_K = 1.380649e-16     # erg/K
sigma_T = 6.6524587321e-25      # cm^2
m_p = 1.67262192369e-24         # g
Mpc_cm = 3.085677581491367e24   # cm

# ---------- fiducial model parameters ----------
xi_gamma = 0.3     # prompt gamma efficiency
xi_L = 0.1         # GeV radiative efficiency (shock -> LAT band)
r0 = 1.0e6         # cm (r0,6 = 1)

# ---------- prompt table (from updated Table 1 with Aaron's T90 corrections) ----------
# E_gamma_45 is E_gamma_iso / 1e45 erg
# T90 values updated: 180128A=0.155s (Trigg+2024), 231024A=0.094s (Trigg+2025 pop),
#                      200423A=0.032s (GBM catalog), 231115A=0.097s (Trigg+2025 M82)
events = [
    dict(GRB="081213A", trig="bn081213174", host="NGC 253", d_Mpc=3.7, T90_s=0.050, Ep_keV=400, alpha=-0.6, E_gamma_45=0.42),
    dict(GRB="180128A", trig="bn180128391", host="NGC 253", d_Mpc=3.7, T90_s=0.155, Ep_keV=290, alpha=-0.6, E_gamma_45=0.60),
    dict(GRB="200415A", trig="bn200415852", host="NGC 253", d_Mpc=3.7, T90_s=0.200, Ep_keV=1080, alpha=-0.0, E_gamma_45=14.2),
    dict(GRB="231024A", trig="bn231024477", host="NGC 253", d_Mpc=3.7, T90_s=0.094, Ep_keV=500, alpha=-0.8, E_gamma_45=0.55),
    dict(GRB="120616A", trig="bn120616630", host="IC 342",  d_Mpc=2.3, T90_s=0.050, Ep_keV=500, alpha=-0.4, E_gamma_45=0.23),
    dict(GRB="200423A", trig="bn200423397", host="NGC 6946", d_Mpc=7.7, T90_s=0.032, Ep_keV=600, alpha=-0.7, E_gamma_45=8.50),
    dict(GRB="231115A", trig="bn231115357", host="M82",     d_Mpc=3.5, T90_s=0.097, Ep_keV=600, alpha=-0.1, E_gamma_45=1.15),
]

# ---------- LAT inputs you can plug in ----------
# Use 1000 s-window *energy flux* ULs (0.1-10 GeV) in units of 1e-9 erg cm^-2 s^-1.
# If "unconstrained", set to None.
# IMPORTANT: if your likelihood fit used a GTI shorter than 1000 s, set T_eff_s accordingly.
lat_1000s = {
    "081213A": dict(FE_ul_1e_9=2.75, T_eff_s=1000),  # change T_eff_s to GTI if you want
    "180128A": dict(FE_ul_1e_9=0.85, T_eff_s=1000),
    "200415A": dict(FE_ul_1e_9=None, T_eff_s=1000),  # detected; handle separately if desired
    "231024A": dict(FE_ul_1e_9=1.29, T_eff_s=1000),
    "120616A": dict(FE_ul_1e_9=None, T_eff_s=1000),  # unconstrained for early window per your choice
    "200423A": dict(FE_ul_1e_9=1.99, T_eff_s=1000),
    "231115A": dict(FE_ul_1e_9=1.05, T_eff_s=1000),
}

def fireball_derived(row):
    # Prompt energy and luminosity
    E_gamma_iso = row["E_gamma_45"] * 1e45  # erg
    L_gamma_iso = E_gamma_iso / row["T90_s"]  # erg/s
    L_gamma_47 = L_gamma_iso / 1e47

    # Total fireball luminosity
    L0 = L_gamma_iso / xi_gamma

    # T0 in keV:
    # T[K] = (L0 / (4pi r0^2 c a))^(1/4)
    T0_K = (L0 / (4*np.pi * r0**2 * c * a_rad))**0.25
    T0_keV = (kB_erg_per_K * T0_K) / 1.602176634e-9  # erg -> keV (1 keV = 1.602e-9 erg)

    # eta_*:
    eta_star = (L0 * sigma_T / (4*np.pi * m_p * c**3 * r0))**0.25

    # baryon-poor mass limit Mb^(BP)
    Mb_BP = E_gamma_iso / (xi_gamma * eta_star * c**2)  # g

    # baryon-rich kinetic estimate Ek_iso
    Ek_iso = E_gamma_iso * (1/xi_gamma - 1)  # erg

    return dict(
        E_gamma_iso=E_gamma_iso,
        L_gamma_iso=L_gamma_iso,
        L_gamma_47=L_gamma_47,
        T0_keV=T0_keV,
        eta_star=eta_star,
        Mb_BP_g=Mb_BP,
        Mb_BP_1e22=Mb_BP/1e22,
        Ek_iso_erg=Ek_iso,
        Ek_iso_1e46=Ek_iso/1e46
    )

def elat_iso_from_energy_flux(d_Mpc, FE_erg_cm2_s, T_eff_s):
    d_cm = d_Mpc * Mpc_cm
    return 4*np.pi * d_cm**2 * FE_erg_cm2_s * T_eff_s  # erg

rows_out = []
for ev in events:
    base = ev.copy()
    der = fireball_derived(ev)
    base.update(der)

    # LAT-based quantities (1000 s window by default)
    lat = lat_1000s.get(ev["GRB"], None)
    if lat and (lat["FE_ul_1e_9"] is not None):
        FE = lat["FE_ul_1e_9"] * 1e-9  # erg cm^-2 s^-1
        T_eff = lat["T_eff_s"]
        E_lat_iso = elat_iso_from_energy_flux(ev["d_Mpc"], FE, T_eff)
        Mb_LAT = E_lat_iso / (xi_L * base["eta_star"] * c**2)  # g
        base.update(dict(
            E_LAT_iso_erg=E_lat_iso,
            E_LAT_iso_1e46=E_lat_iso/1e46,
            Mb_LAT_g=Mb_LAT,
            Mb_LAT_1e22=Mb_LAT/1e22,
            T_eff_s=T_eff
        ))
    else:
        base.update(dict(
            E_LAT_iso_erg=None,
            E_LAT_iso_1e46=None,
            Mb_LAT_g=None,
            Mb_LAT_1e22=None,
            T_eff_s=lat["T_eff_s"] if lat else None
        ))

    rows_out.append(base)

df = pd.DataFrame(rows_out)

# Keep the key columns you care about for Table 3 checks
cols = [
    "GRB","d_Mpc","T90_s","E_gamma_45","L_gamma_47","T0_keV","eta_star",
    "Mb_BP_1e22","Ek_iso_1e46","E_LAT_iso_1e46","Mb_LAT_1e22","T_eff_s"
]
print(df[cols].to_string(index=False))

# If you want: export for easy pasting
df[cols].to_csv("derived_params_check.csv", index=False)
print("\nWrote derived_params_check.csv")

In [2]:
import numpy as np

GEV2ERG = 1.602176634e-3

def K_from_photon_flux(Fph, gamma, Emin, Emax):
    """
    Given integrated photon flux Fph over [Emin,Emax] (GeV),
    for dN/dE = K E^gamma, return K in ph cm^-2 s^-1 GeV^-1.
    """
    if np.isclose(gamma, -1.0):
        return Fph / np.log(Emax / Emin)
    return Fph * (gamma + 1.0) / (Emax**(gamma + 1.0) - Emin**(gamma + 1.0))

def energy_flux_from_K(K, gamma, Emin, Emax):
    """
    Energy flux over [Emin,Emax] (GeV) in erg cm^-2 s^-1.
    """
    if np.isclose(gamma, -2.0):
        F_GeV = K * np.log(Emax / Emin)
    else:
        F_GeV = K * (Emax**(gamma + 2.0) - Emin**(gamma + 2.0)) / (gamma + 2.0)
    return F_GeV * GEV2ERG

def photon_flux_in_band_from_K(K, gamma, Emin, Emax):
    """
    Photon flux over [Emin,Emax] (GeV) in ph cm^-2 s^-1.
    """
    if np.isclose(gamma, -1.0):
        return K * np.log(Emax / Emin)
    return K * (Emax**(gamma + 1.0) - Emin**(gamma + 1.0)) / (gamma + 1.0)

def convert_ul(Fph_ul_in, gamma=-2.0,
               band_in=(0.1, 100.0),
               band_energy_out=(0.1, 10.0),
               band_photon_out=(0.1, 10.0)):
    """
    Convert an integrated photon-flux UL (in band_in) to:
      - photon-flux UL in band_photon_out
      - energy-flux UL in band_energy_out
    """
    Ein1, Ein2 = band_in
    K = K_from_photon_flux(Fph_ul_in, gamma, Ein1, Ein2)

    Fph_out = photon_flux_in_band_from_K(K, gamma, *band_photon_out)
    FE_out  = energy_flux_from_K(K, gamma, *band_energy_out)
    return K, Fph_out, FE_out

# --- your two current stacking ULs (from the summary lines) ---
Fph_ul_100s  = 4.91e-7   # ph/cm^2/s in 0.1–100 GeV
Fph_ul_1000s = 2.51e-7   # ph/cm^2/s in 0.1–100 GeV

for label, Fph in [("100 s", Fph_ul_100s), ("1000 s", Fph_ul_1000s)]:
    K, Fph_0p1_10, FE_0p1_10 = convert_ul(Fph, gamma=-2.0,
                                         band_in=(0.1,100.0),
                                         band_photon_out=(0.1,10.0),
                                         band_energy_out=(0.1,10.0))
    print(label)
    print("  K =", K, "ph cm^-2 s^-1 GeV^-1")
    print("  photon UL (0.1–10) =", Fph_0p1_10, "ph cm^-2 s^-1")
    print("  energy UL (0.1–10) =", FE_0p1_10, "erg cm^-2 s^-1")


100 s
  K = 4.914914914914915e-08 ph cm^-2 s^-1 GeV^-1
  photon UL (0.1–10) = 4.865765765765766e-07 ph cm^-2 s^-1
  energy UL (0.1–10) = 3.626369738922448e-10 erg cm^-2 s^-1
1000 s
  K = 2.5125125125125125e-08 ph cm^-2 s^-1 GeV^-1
  photon UL (0.1–10) = 2.4873873873873874e-07 ph cm^-2 s^-1
  energy UL (0.1–10) = 1.8538061190825549e-10 erg cm^-2 s^-1


In [3]:
import numpy as np

GEV_TO_ERG = 1.602176634e-3

def photonflux_to_K(phi, emin, emax, gamma=-2.0):
    """phi in ph/cm^2/s over [emin,emax] with E in GeV."""
    if np.isclose(gamma, -1.0):
        raise ValueError("gamma=-1 needs log integral for photon flux.")
    return phi * (gamma + 1.0) / (emax**(gamma+1.0) - emin**(gamma+1.0))

def K_to_photonflux(K, emin, emax, gamma=-2.0):
    if np.isclose(gamma, -1.0):
        return K * np.log(emax/emin)
    return K * (emax**(gamma+1.0) - emin**(gamma+1.0)) / (gamma+1.0)

def K_to_energyflux_erg(K, emin, emax, gamma=-2.0):
    """Returns energy flux in erg/cm^2/s over [emin,emax]."""
    if np.isclose(gamma, -2.0):
        f_gev = K * np.log(emax/emin)              # GeV/cm^2/s
    else:
        f_gev = K * (emax**(gamma+2.0) - emin**(gamma+2.0)) / (gamma+2.0)
    return f_gev * GEV_TO_ERG

def convert_phi_0p1_100_to_Ferg_0p1_10(phi_0p1_100, gamma=-2.0):
    # infer K from 0.1–100 photon flux, then compute 0.1–10 energy flux
    K = photonflux_to_K(phi_0p1_100, emin=0.1, emax=100.0, gamma=gamma)
    Ferg_0p1_10 = K_to_energyflux_erg(K, emin=0.1, emax=10.0, gamma=gamma)
    phi_0p1_10 = K_to_photonflux(K, emin=0.1, emax=10.0, gamma=gamma)
    return phi_0p1_10, Ferg_0p1_10

# Example: your stacked ULs
for label, phi_ul in [("100s", 5.09e-06), ("1000s", 2.51e-07), ("10000s", 4.31e-08)]:
    phi10, Ferg10 = convert_phi_0p1_100_to_Ferg_0p1_10(phi_ul, gamma=-2.0)
    print(label, "phi_UL(0.1-10)=", phi10, "F_UL(0.1-10)=", Ferg10)


100s phi_UL(0.1-10)= 5.044144144144144e-06 F_UL(0.1-10)= 3.7593120104104396e-09
1000s phi_UL(0.1-10)= 2.4873873873873874e-07 F_UL(0.1-10)= 1.8538061190825549e-10
10000s phi_UL(0.1-10)= 4.271171171171171e-08 F_UL(0.1-10)= 3.1832288339624744e-11
